In [0]:
!pip install kaggle
!pip install kagglehub[pandas-datasets]

In [0]:
import delta
import kagglehub
from kagglehub import KaggleDatasetAdapter
from pyspark import SparkContext

def table_exists(database, table):
    count = (spark.sql(f"SHOW TABLES FROM {database}")
                  .filter(f"database='{database}' AND tableName='{table}'")
                  .count())    
    return count == 1

In [0]:
database = "bronze"
tablename = "customers"
id_field = "idCliente"
timestamp_field = "DtAtualizacao"

In [0]:
# O arquivo CSV não possui schea, então foi passado alguns parâmetros para ele definir. 
# Já com arquivos Parquet,  naturalmente já vem o schema inferido pois possui metadados.
df_full = spark.read.format("csv").options(sep=";", header=True).load(f"/Volumes/workspace/upsell/full_load/{tablename}/")
schema = df_full.schema

In [0]:
if not table_exists(database, tablename):
    print("Tabela não existente, criando tabela...")
    df_full = spark.read.format("csv").options(sep=";", header=True).load(f"/Volumes/workspace/upsell/full_load/{tablename}/")
    (df_full.coalesce(1).write.format("delta").mode("overwrite").saveAsTable(f"{schema}.{tablename}"))
else:
    print("Tabela já existente, ignorando a carga completa.")

### Atualização da tabela - ReadStream

In [0]:
path = kagglehub.dataset_download("teocalvo/teomewhy-loyalty-system")
print(f"Dataset baixado em: {path}")

# Definir destino dos arquivos
dest_path =  "/Volumes/workspace/upsell/full_load/kaggle"
os.makedirs(dest_path, exist_ok=True)

# Copiando arquivos para o diretório de destino
for file in os.listdir(path):
    if file.startswith("clientes") and file.endswith(".csv"):
        shutil.copy(os.path.join(path, file), dest_path)

In [0]:
df_teste = spark.read.format("csv").options(sep=";", header=True).load(f"/Volumes/workspace/upsell/full_load/kaggle")

display(df_teste)

In [0]:
bronze = delta.DeltaTable.forName(spark, f"{schema}.{tablename}")

def upsert(df, deltatable):
  df.createOrReplaceGlobalTempView(f"view_{tablename}")

  query = f'''
      SELECT * 
      FROM global_temp.view_{tablename}
      QUALIFY ROW_NUMBER() OVER (PARTITION BY {id_field} ORDER BY {timestamp_field} DESC) = 1
  '''

  df_cdc = spark.sql(query)

  (deltatable.alias("b")
             .merge(df_cdc.alias("d"), f"b.{id_field} = d.{id_field}")
             .whenMatchedDelete(condition = "d.OP = 'D'")
             .whenMatchedUpdateAll(condition = "d.OP = 'U'")
             .whenNotMatchedInsertAll(condition = "d.OP = 'I' or d.OP = 'U'")
             .execute()
  )

# Dataframe que realiza a leitura dos dados no formato stream.
df_stream = (spark.readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "parquet")
  #.option("cloudFiles.maxFilesPerTrigger", 500)
  .schema(schema)
  .load(f"/Volumes/workspace/upsell/cdc/{tablename}/"))

# Etapa que realiza a persistência dos dados.
# Para cada batch recebido, ele aplicará uma função chamada upsert que recebe o pedaço de dados e a base onde será salvo.
stream = (df_stream.writeStream
          .option("checkpointLocation", f"/Volumes/workspace/upsell/cdc/{tablename}_checkpoint/")
          .foreachBatch(lambda df, batchID: upsert(df, bronze))
          .trigger(availableNow=True)
        )


In [0]:
start = stream.start()

In [0]:
%sql

SELECT * FROM bronze.customers